In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May 15 17:22:08 2017

@author: cnbyb
"""

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1
tf.Session(config=config)
import numpy as np

def g(x):
    return tf.nn.relu(x)**3
    #return 1.0507*(tf.nn.relu(x)+1.6733*tf.exp(-tf.nn.relu(-x))-1.6733)
    
def block(x,size,scope):
    initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.1)
    with tf.variable_scope(scope):
        W1=tf.get_variable('w1',shape=size,dtype='float',initializer=initializer)
        b1=tf.get_variable('b1',dtype='float',initializer=tf.zeros([size[1]]))
        W2=tf.get_variable('w2',shape=[size[1],size[1]],dtype='float',initializer=initializer)
        b2=tf.get_variable('b2',dtype='float',initializer=tf.zeros([size[1]]))
        y=tf.matmul(x,W1)+b1
        y=g(y)
        y=tf.matmul(y,W2)+b2
        
        if (size[0]<size[1]):
            y=g(y)+tf.concat([x,tf.matmul(x,tf.zeros([2,size[1]-size[0]]))],axis=-1)
        elif (size[0]==size[1]):
            y=g(y)+x
        else:
            w_res=tf.get_variable('w_res',shape=[size[0],size[1]],dtype=tf.float32)
            y=g(y)+tf.matmul(x,w_res)
        
    return y

def func(x,is_training):
    size=10
    y=block(x,[2,size],'block1')
    y=block(y,[size,size],'block2')
    y=block(y,[size,size],'block3')
    y=block(y,[size,size],'block4')
    y=block(y,[size,size],'block5')
    #y=block(y,[size,size],'block6')
    #y=block(y,[size,size],'block7')
    #y=block(y,[size,size],'block8')
    W=tf.get_variable("w", shape=[size,1], dtype='float')
    b=tf.get_variable("b", shape=[1], dtype='float')
    u=tf.matmul(y,W)+b
    return u
def pfunc(x):
    return x[:,0]*x[:,1]

#sample node points in Omega, nodeb*4 points on the border of Omega
def get_batch(node,nodeb):
    x=np.random.random([node,2])
    xblist=[]
    for d in range(2):
        xb0=np.random.random([nodeb,2])
        xb0[:,d]=0.0
        xb1=np.random.random([nodeb,2])
        xb1[:,d]=1.0
        xblist.append(xb0)
        xblist.append(xb1)
    return x,np.concatenate(xblist,axis=0)

node=500
nodeb=100
#x=tf.placeholder(tf.float32,[(2*node+1)**2,2])
x=tf.placeholder(tf.float32,[None,2])
#xb=tf.placeholder(tf.float32,[nodeb*5,2])
xb=tf.placeholder(tf.float32,[None,2])
is_training=tf.placeholder(tf.bool)
with tf.variable_scope('y') as scope:
    u=func(x,is_training)
    scope.reuse_variables()
    ub=func(xb,is_training)
pub=pfunc(xb)
pu=pfunc(x)
ux=tf.gradients(u,x)[0]
#beta=tf.get_variable(name='beta',1e2,trainable=False)
beta=5e2
Iusum=tf.reduce_mean(tf.reduce_sum(ux*ux,axis=1))/2
loss=Iusum+beta*tf.reduce_mean((ub[:,0]-pub)**2)
global_step=tf.get_variable(name='global_step',initializer=0,trainable=False)
lr=tf.train.exponential_decay(1e-2,global_step,10000,0.5,staircase=True)
op=tf.assign_add(global_step,1)
#opt=tf.train.AdamOptimizer(lr,0.99)
opt=tf.train.AdamOptimizer(lr)
with tf.control_dependencies([op]):
    grad=opt.compute_gradients(loss)
    train_op=opt.apply_gradients(grad)
sess=tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(50000):
    x_batch,xb_batch=get_batch(node,nodeb)
    sess.run(train_op,feed_dict={x:x_batch,xb:xb_batch,is_training:True})
    if ((i%100)==0):
        #每一百次输出：迭代次数；I(u)减去正则项的值；I(u)的值
        print(i,sess.run([Iusum,loss],feed_dict={x:x_batch,xb:xb_batch,is_training:False}))
node=10240
xgrid=np.random.random([node,2])

u_solve=sess.run(u,feed_dict={x:xgrid,is_training:False})
pu=pfunc(xgrid)
#pu=np.reshape(pfunc_new(xgrid),[2*node+1,2*node+1])

#相对误差
print(np.linalg.norm(np.reshape(u_solve[:,0]-pu,[-1]),ord=2)/np.linalg.norm(np.reshape(pu,[-1]),ord=2))




D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


KeyboardInterrupt: 